# Сбор и разметка данных

## Урок 9. Инструменты разметки наборов данных


In [1]:
import os
import wget

import pandas as pd

from sklearn.model_selection import train_test_split

import nltk
# Загружаем предобученную модель токенизации текста
nltk.download('punkt_tab')

from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Barsonchik\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


### Практическое задание

**Задание 1.**
Выберите датасет, который имеет отношение к вашей области интересов или исследований. Датасет должен содержать неструктурированные данные, требующие разметки для решения конкретной задачи, например, анализа настроений или распознавания именованных сущностей.

Возьмем датасет новостей rbc.ru

In [2]:
df = pd.read_csv('articles.csv', engine='python', on_bad_lines='skip')
df.head(10)

,id,project,project_nick,type,category,title,body,publish_date,publish_date_t,fronturl,picture,badge,pay_option,data,_score,overview,text
0,6721d7ee9a79471dad034abd,РБК,rbcnews,short_news,Финансы,«Известия» напомнили о налоге на доходы от вкл...,NaN,2024-10-30T10:16:41+03:00,1730272601,https://www.rbc.ru/rbcfreenews/6721d7ee9a79471...,https://s0.rbk.ru/v6_top_pics/media/img/1/75/3...,NaN,free,NaN,1,NaN,Под налог по итогам 2024 года попадут доходы о...
1,6721c7b59a7947f5e247c505,РБК,rbcnews,article,Спорт,Овечкин впервые в сезоне сделал дубль и сократ...,Российский форвард Александр Овечкин впервые в...,2024-10-30T10:07:19+03:00,1730272039,https://www.rbc.ru/sport/30/10/2024/6721c7b59a...,https://s0.rbk.ru/v6_top_pics/media/img/2/12/3...,NaN,free,NaN,1,Российский форвард Александр Овечкин впервые в...,«Вашингтон Кэпиталз» обыграл «Нью-Йорк Рейндже...
2,6721ce5e9a79470242232f14,РБК,rbcnews,article,Политика,Харрис назвала целью Трампа создание «большего...,Харрис назвала Трампа «неуравновешенным» и ука...,2024-10-30T10:02:20+03:00,1730271740,https://www.rbc.ru/politics/30/10/2024/6721ce5...,https://s0.rbk.ru/v6_top_pics/media/img/4/12/3...,NaN,free,NaN,1,Харрис назвала Трампа «неуравновешенным» и ука...,Республиканец Дональд Трамп стремится к бескон...
3,6721d2c39a7947210be19006,РБК,rbcnews,short_news,Политика,Посольству России не сообщили о задержании жур...,NaN,2024-10-30T09:56:25+03:00,1730271385,https://www.rbc.ru/rbcfreenews/6721d2c39a79472...,https://s0.rbk.ru/v6_top_pics/media/img/0/38/3...,NaN,free,NaN,1,NaN,Власти США не сообщили в посольство России в В...
4,6721cc239a79475d15fe6523,РБК,rbcnews,article,Политика,"NYT узнала, что Израиль нанес удар по крупному...",В ночь на 26 октября Израиль нанес авиаудары п...,2024-10-30T09:43:23+03:00,1730270603,https://www.rbc.ru/politics/30/10/2024/6721cc2...,https://s0.rbk.ru/v6_top_pics/media/img/8/94/3...,NaN,free,NaN,1,В ночь на 26 октября Израиль нанес авиаудары п...,В результате атаки израильских ВВС на Иран в н...
5,6721d2a09a79479e2d706e56,РБК,rbcnews,article,Общество,Шереметьево на час ограничивало полеты,Ухода рейсов на запасные аэродромы не было,2024-10-30T09:31:00+03:00,1730269860,https://www.rbc.ru/society/30/10/2024/6721d2a0...,https://s0.rbk.ru/v6_top_pics/media/img/2/47/3...,NaN,free,NaN,1,Ухода рейсов на запасные аэродромы не было,Столичный аэропорт Шереметьево не принимал рей...
6,6721cc2f9a79470e3835e239,РБК,rbcnews,short_news,Общество,"В Москве выявили схемы, по которым более 100 ч...",NaN,2024-10-30T09:30:21+03:00,1730269821,https://www.rbc.ru/rbcfreenews/6721cc2f9a79470...,https://s0.rbk.ru/v6_top_pics/media/img/3/12/3...,NaN,free,NaN,1,NaN,В Москве возбудили уголовное дело против мошен...
7,6720d63d9a79478690f58b0b,РБК,rbcnews,article,Экономика,В РАНХиГС сообщили о снижении доли государства...,"Доля государства в ВВП сократилась с 51,7 до 4...",2024-10-30T09:30:19+03:00,1730269819,https://www.rbc.ru/economics/30/10/2024/6720d6...,https://s0.rbk.ru/v6_top_pics/media/img/7/11/3...,NaN,free,NaN,1,"Доля государства в ВВП сократилась с 51,7 до 4...",Доля госсектора в экономике России в 2023 году...
8,6720fff19a794704eedc8a70,РБК,rbcnews,article,Спорт,Экс-судья КХЛ выступил за дисквалификацию трен...,КХЛ оштрафовала «Спартак» из-за поведения боле...,2024-10-30T09:14:08+03:00,1730268848,https://www.rbc.ru/sport/30/10/2024/6720fff19a...,https://s0.rbk.ru/v6_top_pics/media/img/7/00/3...,NaN,free,NaN,1,КХЛ оштрафовала «Спартак» из-за поведения боле...,Главного тренера петербургского СКА Романа Рот...
9,6721c5499a79470d23d6121b,РБК,rbcnews,article,Политика,Microsoft выявила атаку хакеров из «Полуночной...,Microsoft обвинила хакерскую группировку Midni...,2024-10-30T09:08:31+03:00,1730268511,https://www.rbc.ru/politics/30/10/2024/6721c54...,https://s0.rbk.ru/v6_top_pics/media/img/4/07/3...,NaN,free,NaN,1,Microsoft обвинила хакерскую группировку Midni...,Российская хакерская группировка Midnight Bliz...


**Задание 2.**
Выполните разметку на основе правил (rule-based labeling) на подмножестве выбранного датасета. Разработайте и реализуйте набор правил или условий, которые позволят автоматически присваивать метки данным на основе определенных шаблонов или критериев.

Загрузим тональный словарь.

In [3]:
url = 'https://raw.githubusercontent.com/dkulagin/kartaslov/master/dataset/kartaslovsent/kartaslovsent.csv'
file = 'kartaslovsent.csv'
if not os.path.isfile("./kartaslovsent.csv"):
    wget.download(url, out=f"./{file}")

In [4]:
df_tone_dic = pd.read_csv('./kartaslovsent.csv', sep=';', on_bad_lines='skip')
df_tone_dic.head()

,term,tag,value,pstv,ngtv,neut,dunno,pstvNgtvDisagreementRatio
0,абажур,NEUT,0.08,0.185,0.037,0.580,0.198,0.00
1,аббатство,NEUT,0.10,0.192,0.038,0.578,0.192,0.00
2,аббревиатура,NEUT,0.08,0.196,0.000,0.630,0.174,0.00
3,абзац,NEUT,0.00,0.137,0.000,0.706,0.157,0.00
4,абиссинец,NEUT,0.28,0.151,0.113,0.245,0.491,0.19


Перемешаем и разделим датафрейм на две части (1-разметка вручную, 2-разметка на основе правил).

In [5]:
title1, title2 = train_test_split(df['title'], train_size=5, random_state=42)

In [6]:
df_title1 = pd.DataFrame(title1)

In [7]:
df_title1.head()

,title
12,Апелляционный суд оставил в силе приговор «кра...
7,В РАНХиГС сообщили о снижении доли государства...
10,Пожар на складе удобрений в Долгопрудном полно...
14,Губернатор Курской области рассказал о последс...
6,"В Москве выявили схемы, по которым более 100 ч..."


In [8]:
df_title2 = pd.DataFrame(title2)

In [9]:
df_title2.head()

,title
0,«Известия» напомнили о налоге на доходы от вкл...
17,"NYT узнала, как США на украинском опыте готовя..."
15,Жительница Курской области погибла при обстрел...
1,Овечкин впервые в сезоне сделал дубль и сократ...
8,Экс-судья КХЛ выступил за дисквалификацию трен...


In [10]:
# Функция для присвоения метки настроения на основе наличия положительных или отрицательных слов
def get_sentiment(text, df_td=df_tone_dic):
    # Токенизация текста на отдельные слова
    words = nltk.word_tokenize(text.lower())
    sum_value = 0  # Сумма всех зачений весов найденных слов

    for word in words:
        if len(df_td[df_td.term == word]):
            df_filtered = df_td[df_td['term'] == word]
            value = df_filtered['value'].iloc[0]
            sum_value += value

    return sum_value

In [11]:
# Функция для присвоения метки настроения на основе оценки полярности настроения
def get_sentiment_label(score):
    if score < -0.35:
        return 'negative'
    elif score >= 0.55:
        return 'positive'
    else:
        return 'neutral'

In [12]:
print(get_sentiment('Как хорошо, что я успел вовремя'))
print(get_sentiment('Удар молнии'))
print(get_sentiment('Всё нормально'))

1.33
-0.68
0


In [13]:
print(get_sentiment_label(get_sentiment('Как хорошо, что я успел вовремя')))
print(get_sentiment_label(get_sentiment('Удар молнии')))
print(get_sentiment_label(get_sentiment('Всё нормально')))

positive
negative
neutral


Присвоим метки настроения на основе правил.

In [14]:
df_title2['sentiment'] = df_title2['title'].apply(get_sentiment)
df_title2['sentiment_label'] = df_title2['sentiment'].apply(get_sentiment_label)

**Задача 3.**
Выполните разметку вручную отдельного подмножества выбранного датасета с помощью выбранного вами инструмента разметки.

Сохраним датафрем в CSV файл.

In [15]:
df_title1.to_csv('manual_marking.csv', index=False)

Запускаем локально label-studio и делаем разметку. \
Экспортируем размеченный вручную CSV файл и загружаем его в датафрейм.

In [22]:
df_title1 = pd.read_csv('project-2-at-2024-10-30-07-38-c06cfc4f.csv', engine='python', on_bad_lines='skip')
df_title1.head(20)

,annotation_id,annotator,created_at,id,lead_time,sentiment,title,updated_at
0,10,2,2024-10-30T07:34:01.830748Z,6,9.877,positive,Апелляционный суд оставил в силе приговор «кра...,2024-10-30T07:36:33.159765Z
1,9,2,2024-10-30T07:33:50.288514Z,7,7.099,positive,В РАНХиГС сообщили о снижении доли государства...,2024-10-30T07:38:43.713070Z
2,6,2,2024-10-30T07:33:24.479097Z,8,9.142,negative,Пожар на складе удобрений в Долгопрудном полно...,2024-10-30T07:33:24.479116Z
3,7,2,2024-10-30T07:33:31.965168Z,9,1.889,negative,Губернатор Курской области рассказал о последс...,2024-10-30T07:33:31.965191Z
4,8,2,2024-10-30T07:33:39.267219Z,10,1.954,negative,"В Москве выявили схемы, по которым более 100 ч...",2024-10-30T07:33:39.267240Z


In [23]:
df_title1 = df_title1[['title', 'sentiment']].rename(columns={'sentiment': 'sentiment_label'})
df_title1.head()

,title,sentiment_label
0,Апелляционный суд оставил в силе приговор «кра...,positive
1,В РАНХиГС сообщили о снижении доли государства...,positive
2,Пожар на складе удобрений в Долгопрудном полно...,negative
3,Губернатор Курской области рассказал о последс...,negative
4,"В Москве выявили схемы, по которым более 100 ч...",negative


**Задача 4.**
Объедините данные, размеченные вручную, с данными, размеченными на основе правил. Объедините два подмножества размеченных данных в один набор данных, сохранив при этом соответствующую структуру и целостность.

In [24]:
df_title = pd.concat([df_title1, df_title2], axis=0).reset_index(drop=True)
df_title.drop(columns=['sentiment'], inplace=True)
df_title.head(20)

,title,sentiment_label
0,Апелляционный суд оставил в силе приговор «кра...,positive
1,В РАНХиГС сообщили о снижении доли государства...,positive
2,Пожар на складе удобрений в Долгопрудном полно...,negative
3,Губернатор Курской области рассказал о последс...,negative
4,"В Москве выявили схемы, по которым более 100 ч...",negative
5,«Известия» напомнили о налоге на доходы от вкл...,neutral
6,"NYT узнала, как США на украинском опыте готовя...",neutral
7,Жительница Курской области погибла при обстрел...,neutral
8,Овечкин впервые в сезоне сделал дубль и сократ...,negative
9,Экс-судья КХЛ выступил за дисквалификацию трен...,neutral


In [20]:
df_title['sentiment_label'].value_counts()

sentiment_label
neutral     12
positive     4
negative     4
Name: count, dtype: int64

**Задача 5.**
Обучите модель машинного обучения, используя объединенный набор размеченных данных. Разделите датасет на обучающий и тестовый наборы и используйте обучающий набор для обучения модели.

In [25]:
le = LabelEncoder()
df_title['sentiment_label'] = le.fit_transform(df_title['sentiment_label'].astype(str))
df_title.head()

,title,sentiment_label
0,Апелляционный суд оставил в силе приговор «кра...,2
1,В РАНХиГС сообщили о снижении доли государства...,2
2,Пожар на складе удобрений в Долгопрудном полно...,0
3,Губернатор Курской области рассказал о последс...,0
4,"В Москве выявили схемы, по которым более 100 ч...",0


In [26]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Предполагается, что df_title уже загружен и содержит столбец 'sentiment_label'
le = LabelEncoder()
df_title['sentiment_label'] = le.fit_transform(df_title['sentiment_label'].astype(str))

# Сохранение DataFrame в файл test.csv
df_title.to_csv('test.csv', index=False)

In [27]:
# перемешаем данные
df_title = shuffle(df_title)

In [28]:
# разделим датасет на размеченную и неразмеченную выборки
labeled_title, unlabeled_title = train_test_split(df_title, train_size=0.2, random_state=42)

In [29]:
# функция для обучения модели логистической регрессии на размеченных данных
def train_model(labeled_title):
    # Векторизация текстовых данных с помощью TF-IDF
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(labeled_title['title'])
    y = labeled_title['sentiment_label']

    # Обучение модели логистической регрессии на размеченных данных
    model = LogisticRegression()
    model.fit(X, y)

    return model, vectorizer

In [30]:
# обучение начальной модели на небольшом наборе
model, vectorizer = train_model(labeled_title)

In [31]:
# использование исходной модели для прогнозирования настроения неразмеченных данных
X_unlabeled = vectorizer.transform(unlabeled_title['title'])
y_unlabeled_predicted = model.predict(X_unlabeled)

In [32]:
# вычисление неопределенности или энтропии предсказаний
y_unlabeled_proba = model.predict_proba(X_unlabeled)
uncertainty = -(y_unlabeled_proba * np.log2(y_unlabeled_proba)).sum(axis=1)

In [33]:
# выбор 100 наиболее неопределенных точек данных для маркировки человеком
labeled_title_new = unlabeled_title.iloc[uncertainty.argsort()[:100]]
unlabeled_title_new = unlabeled_title.iloc[uncertainty.argsort()[100:]]

In [34]:
# разметка новых точек данных и добавление их к размеченному множеству
labeled_title = pd.concat([labeled_title, labeled_title_new])
labeled_title.shape

(20, 2)

In [35]:
# переобучение модели на расширенном маркированном множестве
model, vectorizer = train_model(labeled_title)

**Задача 6.**
Оценить эффективность обученной модели на тестовом датасете. Используйте подходящие метрики оценки. Интерпретируйте результаты и проанализируйте эффективность модели в решении задачи разметки.

In [36]:
### Загружаем тестовый датасет
title_test = pd.read_csv('test.csv', engine='python', on_bad_lines='skip')

In [37]:
title_test['sentiment_label'] = le.fit_transform(title_test['sentiment_label'].astype(str))
title_test.head(20)

,title,sentiment_label
0,Апелляционный суд оставил в силе приговор «кра...,2
1,В РАНХиГС сообщили о снижении доли государства...,2
2,Пожар на складе удобрений в Долгопрудном полно...,0
3,Губернатор Курской области рассказал о последс...,0
4,"В Москве выявили схемы, по которым более 100 ч...",0
5,«Известия» напомнили о налоге на доходы от вкл...,1
6,"NYT узнала, как США на украинском опыте готовя...",1
7,Жительница Курской области погибла при обстрел...,1
8,Овечкин впервые в сезоне сделал дубль и сократ...,0
9,Экс-судья КХЛ выступил за дисквалификацию трен...,1


In [38]:
X_test = vectorizer.transform(title_test['title'])
y_test_predicted = model.predict(X_test)

In [39]:
import numpy as np

# Печать уникальных значений для sentiment_label
print(title_test['sentiment_label'].unique())

# Печать уникальных значений для y_test_predicted
print(np.unique(y_test_predicted))

[2 0 1]
[0 1]


В качестве критерия качества модели выберем метрику F1-measure. Она отражает точностью (Precision) и полноту (Recall) и рассчитывается как их среднее гармоническое.

$Precision = \cfrac{TP}{TP + FP}$
<br><br>

$Recall = \cfrac{TP}{TP + FN}$
<br><br>

$F_1 = \cfrac{2}{\cfrac{1}{Recall} + \cfrac{1}{Precision}} = 2 \cfrac{Recall \cdot Precision}{Recall + Precision} = \cfrac{TP}{TP + \cfrac{FP + FN}{2}}$

In [40]:
f1 = f1_score(title_test['sentiment_label'], y_test_predicted, average='micro')
print(f1)

0.75


Эффективность обученной модели составила 75%.

Разметка на основе правил существенно экономит время и увеличивает производительность, но не всегда положительно влияет на качество набора данных.